# インポート

In [ ]:
%reload_ext autoreload

# ROOT_DIRの設定
import sys
import os
# Notebook のディレクトリ（`notebook.ipynb` がある場所）から 2 つ上のディレクトリを取得
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
# sys.path に追加
sys.path.append(parent_dir)

import pandas as pd

from common.src import scraping
from common.src import create_rawdf
from common.src import constants

# データ加工

### レース結果テーブルの加工

In [ ]:
df = pd.read_csv(constants.RACE_RESULTS_RAWDF_DIR / "race_results_2023.csv", sep="\t")
len(df)
df.columns = df.columns.str.replace(r"\s", "", regex=True)

In [ ]:
df.iloc[:, 0:10]

### dtype: objectは機械学習モデルに突っ込めないため変換する

In [ ]:
# 着順 
# df["着順"].astype(int) // errormu

In [ ]:
# 着順 '中止'などを省く
df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df["rank"].value_counts(dropna=False)

In [ ]:
# 欠損値になっている行は　省く
df.dropna(subset=["rank"], inplace=True) # inplace option = df自体を破壊するかどうか

In [ ]:
df["rank"].value_counts(dropna=False)

In [ ]:
df["枠番"].astype(int)

In [ ]:
df["馬番"].astype(int)

### 性齢に関してはラベルエンコーディングを行う
ラベルエンコーディング: カテゴリ変数を整数にマッピングするデータ加工手法

In [ ]:
# 性別
df["性齢"].str[0]

In [ ]:
sex_mapping = { "牡": 0, "牝": 1, "セ": 2 }
df["性齢"].str[0].map(sex_mapping).value_counts()

In [ ]:
# 年齢 ←２文字目以降で取り出せるが、正規表現とかもあり
df["性齢"].str[1:].value_counts()

In [ ]:
df["性齢"].str[1:].astype(int)

In [ ]:
df["斤量"]

In [ ]:
df["単勝"].astype(float)

In [ ]:
df["人気"].astype(int)

In [ ]:
# 場体重のみ取り出す
df["weight"] = df["馬体重"].str.extract(r"(\d+)").astype(int)

In [ ]:
# 場体重変化を取り出す
df["weight_diff"] = df["馬体重"].str.extract(r"\((.+)\)").astype(int)